# K-Means

In this notebook you will use GPU-accelerated K-means to find the best locations for a fixed number of humanitarian supply airdrop depots.

## Objectives

By the time you complete this notebook you will be able to:

- Use GPU-accelerated K-means
- Use cuXfilter to visualize K-means clusters

## Imports

For the first time we import `cuml`, the RAPIDS GPU-accelerated library containing many common machine learning algorithms. We will be visualizing the results of your work in this notebook, so we also import `cuxfilter`.

In [1]:
import cudf
import cuml

import cuxfilter as cxf

## Load Data

For this notebook we load again the cleaned UK population data--in this case, we are not specifically looking at counties, so we omit that column and just keep the grid coordinate columns.

In [2]:
gdf = cudf.read_csv('./data/pop_2-03.csv', usecols=['easting', 'northing'])
print(gdf.dtypes)
gdf.shape

northing    float64
easting     float64
dtype: object


(58479894, 2)

In [3]:
gdf.head()

,northing,easting
0,515491.5313,430772.1875
1,503572.4688,434685.8750
2,517903.6563,432565.5313
3,517059.9063,427660.6250
4,509228.6875,425527.7813


## K-Means Clustering

The unsupervised K-means clustering algorithm will look for a fixed number *k* of centroids in the data and clusters each point with its closest centroid. K-means can be effective when the number of clusters *k* is known or has a good estimate (such as from a model of the underlying mechanics of a problem).

Assume that in addition to knowing the distribution of the population, which we do, we would like to estimate the best locations to build a fixed number of humanitarian supply depots from which we can perform airdrops and reach the population most efficiently. We can use K-means, setting *k* to the number of supply depots available and fitting on the locations of the population, to identify candidate locations.

GPU-accelerated K-means is just as easy as its CPU-only scikit-learn counterpart. In this series of exercises, you will use it to optimize the locations for 5 supply depots.

## Exercise: Make a `KMeans` Instance for 5 Clusters

`cuml.KMeans()` will initialize a K-means instance. Use it now to initialize a K-means instance called `km`, passing the named argument `n_clusters` set equal to our desired number `5`:

#### Solution

In [4]:
# %load solutions/make_k-means_instance
km = cuml.KMeans(n_clusters=5)


## Exercise: Fit to Population

Use the `km.fit` method to fit `km` to the population's locations by passing it the population data. After fitting, add the cluster labels back to the `gdf` in a new column named `cluster`. Finally, you can use `km.cluster_centers_` to see where the algorithm created the 5 centroids.

#### Solution

In [5]:
# %load solutions/km_fit
km.fit(gdf)
gdf['cluster'] = km.labels_
km.cluster_centers_


,0,1
0,150988.668019,315543.991697
1,294886.581587,439240.631466
2,402934.342187,395973.013652
3,540894.692061,415736.503333
4,180242.130118,530767.081392


## Visualize the Clusters

To help us understand where clusters are located, we make a visualization that separates them, using the same three steps as before.

### Associate a Data Source with cuXfilter

### Define Charts and Widgets

In this case, we have an existing integer column to use with multi-select: `cluster`. We use the same technique to scale the scatterplot, then add a widget to let us select which cluster to look at.

In [7]:
# chart_width = 600
# scatter_chart = cxf.charts.datashader.scatter(x='easting', y='northing', 
#                                               width=chart_width, 
#                                               height=int((gdf['northing'].max() - gdf['northing'].min()) / 
#                                                          (gdf['easting'].max() - gdf['easting'].min()) *
#                                                           chart_width))

# cluster_widget = cxf.charts.panel_widgets.multi_select('cluster')

In [8]:
from pyproj import Proj, Transformer

transform_27700_to_3857 = Transformer.from_crs('epsg:27700', 'epsg:3857')
gdf['dropoff_y'], gdf['dropoff_x'] = transform_27700_to_3857.transform(gdf['northing'].values_host, gdf['easting'].values_host)
gdf['dropoff_x'] = gdf['dropoff_x'] - 7300000
gdf['dropoff_y'] = gdf['dropoff_y'] + 7300000
gdf

,northing,easting,cluster,dropoff_y,dropoff_x
0,515491.5313,430772.1875,3,7.272215e+06,-175065.967923
1,503572.4688,434685.8750,3,7.252243e+06,-167970.303699
2,517903.6563,432565.5313,3,7.276362e+06,-172133.313684
3,517059.9063,427660.6250,3,7.274731e+06,-180393.736869
4,509228.6875,425527.7813,3,7.261438e+06,-183679.791727
...,...,...,...,...,...
58479889,192393.7813,340519.8438,0,6.733417e+06,-331440.063201
58479890,183451.8906,337749.6250,0,6.718840e+06,-336681.056828
58479891,189997.5156,341165.1250,0,6.729409e+06,-330542.647280
58479892,184439.1250,335227.6563,0,6.720657e+06,-340783.395312


In [9]:
cxf_data = cxf.DataFrame.from_dataframe(gdf)

In [10]:
import cuxfilter
from bokeh.tile_providers import get_provider
from matplotlib import pyplot as plt

palette = ["red", "grey", "black", "purple", "pink", "yellow", "brown", "green", "orange", "blue"]
chart1 = cuxfilter.charts.scatter(x='dropoff_x', y='dropoff_y', aggregate_col='cluster', aggregate_fn='mean',
                                                    tile_provider="CartoLight", color_palette=palette, pixel_shade_type='linear')

cluster_widget = cuxfilter.charts.panel_widgets.multi_select('cluster')

### Create and Show the Dashboard

In [11]:
from cuxfilter.layouts import feature_and_double_base

dash = cxf_data.dashboard([chart1], sidebar=[cluster_widget], theme=cuxfilter.themes.rapids, title= 'kmeans DATASET')
dash.app()

Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=280)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=58479894)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='cluster_multi_select', width=400)
            [0] MultiSelect(height=200, options=[0, 1, 2, 3, 4, ''], sizing_mode='stretch_both', value=[''], width=400)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=7, nrows=6, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Scatter plot f..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')

In [12]:
# dash = cxf_data.dashboard(charts=[scatter_chart],sidebar=[cluster_widget], theme=cxf.themes.dark, data_size_widget=True)

In [14]:
chart1.view()

Card(sizing_mode='scale_width', title='Scatter plot f..., width=800)
    [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')

In [15]:
%%js
var host = window.location.host;
element.innerText = "'http://"+host+"'";

<IPython.core.display.Javascript object>

Set `my_url` in the next cell to the value just printed, making sure to include the quotes and ignoring the button (due to this contained cloud environment) as before:

In [16]:
my_url = 'http://ec2-44-206-251-206.compute-1.amazonaws.com'# TODO: Set this value to the print out of the cell above, including the quotes.
dash.show(my_url, port=8789)

Dashboard running at port 8789


Row(sizing_mode='stretch_both')
    [0] Button(button_type='success', name='open cuxfilter d..., sizing_mode='stretch_both')

... and you can run the next cell to generate a link to the dashboard:

In [17]:
%%js
var host = window.location.host;
var url = 'http://'+host+'/lab/proxy/8789/';
element.innerHTML = '<a style="color:blue;" target="_blank" href='+url+'>Open Dashboard</a>';

<IPython.core.display.Javascript object>

In [18]:
dash.stop()

<br>
<div align="center"><h2>Please Restart the Kernel</h2></div>

In [19]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Next

In the next notebook, you will use GPU-accelerated DBSCAN to identify geographically dense clusters of infected people.